In [ ]:
import re
import requests
import time
import json

In [2]:
requests.packages.urllib3.disable_warnings()

In [12]:
KEYWORD = "forêt de rambouillet"

In [13]:
initurl =  f'https://catalogue.bnf.fr/changerPage.do?motRecherche={KEYWORD}&nbResultParPage=1&pageEnCours=1'
r = requests.get(initurl)
time.sleep(0.5)
if r.status_code != 200:
    exit(1)
data = r.text

In [5]:
from bs4 import BeautifulSoup

In [14]:
soup = BeautifulSoup(data, 'html.parser')
nbitem = int(soup.find(id="nbPage").text.strip()[3:].replace(' ',''))
print(nbitem)

281


In [15]:
MAX_SEARCH = 100
result = set()
for index in range(1, int(nbitem / MAX_SEARCH) + 2):
    print(f"searching {index}/{int(nbitem / MAX_SEARCH) + 2}")
    searchurl = f'https://catalogue.bnf.fr/changerPage.do?motRecherche={KEYWORD}&nbResultParPage={MAX_SEARCH}&pageEnCours={index}'
    r = requests.get(searchurl)
    if r.status_code != 200:
        exit(1)
    try:
        data = r.text
        soup = BeautifulSoup(data, 'html.parser')

        for item in soup.find_all('div', {"class": "notice-contenu"}):
            for arkdiv in item.find_all('div', {"class": "notice-synthese"}):
                for ark in arkdiv.find_all('a'):
                    if '/ark:/' in ark['href']:
                        result.add(ark['href'])
        # break
    except Exception as e:
        print(f"Something went wrong: {e}")
        exit(1)
    # time.sleep(0.1)
print(f"there is {len(result)} results found")

searching 1/4
searching 2/4
searching 3/4
there is 281 results found


In [17]:
import concurrent
import concurrent.futures

In [18]:
def request_item(id):
    time.sleep(1)
    url = f"https://catalogue.bnf.fr{id}"
    print(url)
    try:
        r = requests.get(url, verify=False, timeout=30)
    except Exception as e:
        return id

    return r

with concurrent.futures.ThreadPoolExecutor() as executor:
    res = [executor.submit(request_item, id) for id in result]
    concurrent.futures.wait(res)


resultfinal = []

error_second_try = []

for i in range(len(result)):
    element = res[i].result()
    if isinstance(element, str):
        print(f"Error: {element}")
        error_second_try.append(element)
        continue
    if element.status_code != 200:
        print(element.status_code)
        print(res.history[0].url)
        continue
    try:
        temp = {}
        soup = BeautifulSoup(element.text, 'html.parser')
        item = soup.find(id='ancreNotice')
        for p in item.find_all('p'):
            temp[p['id'].strip()] = []
            for sp in p.find_all('span'):
                temp[p['id'].strip()].append(sp.getText().strip())

        resultfinal.append(temp)
    except Exception as e:
        print(f"Something went wrong: {e}")

print("number of errors:", len(error_second_try))
print(f"{len(resultfinal)} finals elements")

https://catalogue.bnf.fr/ark:/12148/cb44246507fhttps://catalogue.bnf.fr/ark:/12148/cb41308903khttps://catalogue.bnf.fr/ark:/12148/cb40715068q

https://catalogue.bnf.fr/ark:/12148/cb31787964s
https://catalogue.bnf.fr/ark:/12148/cb416016685
https://catalogue.bnf.fr/ark:/12148/cb46691304n
https://catalogue.bnf.fr/ark:/12148/cb390153585https://catalogue.bnf.fr/ark:/12148/cb41580252x


https://catalogue.bnf.fr/ark:/12148/cb35351733r
https://catalogue.bnf.fr/ark:/12148/cb36136840w
https://catalogue.bnf.fr/ark:/12148/cb31787967t
https://catalogue.bnf.fr/ark:/12148/cb39363792h
https://catalogue.bnf.fr/ark:/12148/cb411972853
https://catalogue.bnf.fr/ark:/12148/cb41517690t
https://catalogue.bnf.fr/ark:/12148/cb406575536
https://catalogue.bnf.fr/ark:/12148/cb40763127z
https://catalogue.bnf.fr/ark:/12148/cb406530697
https://catalogue.bnf.fr/ark:/12148/cb35351735f
https://catalogue.bnf.fr/ark:/12148/cb34696972m
https://catalogue.bnf.fr/ark:/12148/cb304819144
https://catalogue.bnf.fr/ark:/12148/cb40

In [20]:
with open("bnf.json", "w") as f:
    json.dump(resultfinal, f, indent=4)